In [ ]:
%pylab

# Part II: Gamma-Ray Event Processing & Image Reconstruction

In this lesson, we will focus on converting the gamma-ray interaction data into Compton *cone data*; i.e. computing the cone scatter-axes from the interaction positions and the cone angle from Compton kinematics. After successful generation of the cone data, we will create our first image via backprojection using the `ComptonBackprojection2D` class in the `compton_imaging.py` module.

## Compton Imaging

It is assumed that the student is already familiar with Compton kinematics and the theoretical underpinnings of Compton imaging. The goal is to recover information about the incident direction of gamma-rays interacting in the detector. To do so, we need two pieces of information:

 1. The energy deposited in the Compton scattering interaction
 2. The direction of the scattered gamma-ray

The direction of the scattered gamma-ray is given by the locations of the first two gamma-ray interactions, and forms the axis of the Compton cone. The deposited energy is used to infer the possible incident direction of gamma-rays via the Compton scattering equation:

$$\cos(\theta) = 1 + \frac{1}{A_{0}} - \frac{1}{A_{d}}$$

Where:

$$A_{0} = \frac{E_{0}}{m_{e}}$$ and
$$A_{d} = \frac{E_{0} - E_{dep}}{m_{e}}$$

## Getting Started

We begin by loading and selecting data from our dataset that is suitable for imaging.

In [ ]:
# Load the data
import tables
with tables.open_file("hits.h5", "r") as hf:
    event_pointers = hf.root.EventPointers.read()
    event_lengths = hf.root.EventLengths.read()
    idata = hf.root.InteractionData.read()

In [ ]:
# Create an array of double-interaction events
num_interactions_per_event = 2
evmask = event_lengths == num_interactions_per_event
ptrs = event_pointers[evmask]
doubles = np.array([idata[p:p+num_interactions_per_event] for p in ptrs])

Feel free to create the energy spectrum of your selected double-interaction events to verify that things look okay (N.B. the spectrum should match your results from the previous notebook).

## Full-Energy Events

The Compton kinematic equations shown above require you to know two energy values in order to be able to recover angular information:

 1. The energy deposited by the photon in the initial Compton scatter interaction.
 2. The incident energy of the un-scattered photon.
 
The first value is recorded directly by the detector. However, the incident photon energy is another story. For spectroscopic imaging systems, the most common method for determining the incident photon energy is to set an **energy window** around a spectroscopic feature of interest, most often a full-energy peak.

For example, the photons comprising our dataset were generated from a simulated Cs-137 source, which emits photons with an energy of 661.657 keV. The full-energy peak corresponding to photons that deposit their full energy in the active volume of the detector is clearly visible in the energy spectrum. These are the events we would like to focus on for gamma-ray imaging.

### Exercise 1

From the double-interaction events loaded above, select only the events that deposit the full incident photon energy in the detector. What fraction of the double-interaction events deposit their full energy in the active volume of the detector?

In [ ]:
# Boolean mask in energy
ens = doubles['energy'].sum(axis=1)   # Total energy deposited in both interactions
ppk_mask = (ens >= 661.) & (ens <= 662.)
# Print fraction of full-interaction events
print 'Fraction of double-interaction events that deposit full energy: %.3f' %(ppk_mask.sum()/float(ppk_mask.shape[0]))
# Select the photopeak events
ppk_evs = doubles[ppk_mask]

## Converting Events to Cone Data

At this point, we have a subset of the gamma-ray event data that correspond to the double-interaction events that deposit their full energy in the active volume of the detector. The final step in the event processing is to compute the scatter-axes and cone opening angles for each of the events. 

*N.B. The simulated events are recorded in the order in which they occur, i.e. the first interaction in the gamma-ray event occurred first temporally*

### Exercise 2

Using the full-energy, double-interaction event data, compute the cone opening angle for each gamma-ray event using the kinematic equations given above. *Hint: what is the expected range of cone opening angles? Make sure that your results fall within this range!*

In [ ]:
m_e = 511.    # Electron rest mass, keV
E0 = 661.657  # Incident photon energy, keV (exact)
Ed = ppk_evs['energy'][:,0]   # Energy deposited in first interaction, keV
# Kinematic computations
A0 = E0 / m_e
Ad = (E0 - Ed) / m_e
costh = 1 + (1/A0) - (1/Ad)
th_deg = np.arccos(costh) * (180./np.pi)

Plot the distribution of cone angles. Does the range of angles match your expectations? What is the most probable cone opening angle? What factors affect this distribution?

### Exercise 3

Using the full-energy gamma-ray event data, compute unit-vectors corresponding to the scatter-axes of the Compton cones.

In [ ]:
# Extract the individual vector components from the interaction positions
xd = ppk_evs['x'][:,0] - ppk_evs['x'][:,1]                                    
yd = ppk_evs['y'][:,0] - ppk_evs['y'][:,1]                                    
zd = ppk_evs['z'][:,0] - ppk_evs['z'][:,1]
# Compute the cone-axis unit-vectors
cone_dirs = (np.array([xd, yd, zd]) / np.sqrt(xd**2 + yd**2 + zd**2)).T

#### We will be using these computations in the next lesson as well, so if you have the time, we recommend creating functions to compute the cone axes and opening angles from Compton event data.

## Compton Imaging

The simplest Compton imaging method is to back-project the Compton cones onto some imaging space. One of the unique features of Compton imaging is the very wide field-of-view (FOV) that is achievable. As a result, it is quite common to see Compton backprojections computed on non-planar surfaces such as full or partial spheres to capture the full FOV.

In the interest of time, we have provided you with a Compton backprojection algorithm. The algorithm is encapsulated in the `ComptonBackprojection2D` class that can be found in the `compton_imaging.py` module. The `ComptonBackprojection2D` class handles the tasks of generating a $4\pi$ imaging space, discretizing it into pixels, and computing the backprojection over the discretized space. We won't discuss the algorithm in detail here, but everything from the course is open source (and significant effort has been put towards documentation), so feel free to study, use, and modify the code as you see fit.

In [ ]:
from compton_imaging import ComptonBackprojection2D

For the purposes of the in-person portion of the course, the most important thing is that you understand the interface to the `ComptonBackprojection2D` class. The `ComptonBackprojection2D` class has a method called `backproject_cones`. The docstring for this method should give us an idea of how it is used:

In [ ]:
ComptonBackprojection2D.backproject_cones?

We can see that it expects the cone scatter axes and the cone opening angles that we have computed in the previous exercises as input. *N.B. The expected format for the inputs is given in the docstring!*

If we instantiate a `ComptonBackprojection2D` object without any arguments, a reasonable $4\pi$ imaging space with $1^{\circ}$ discretization is generated by default.

In [ ]:
backprojector = ComptonBackprojection2D()

### Exercise 4

Create a Compton image from the first 100 Compton events in the dataset.

In [ ]:
# Number of events to use to create image
num_cones = 1000
# Select the appropriate number of events
caxs = cone_dirs[:num_cones,...]
cang = th_deg[:num_cones]
# Create image
img = backprojector.backproject_cones(caxs, cang)

In [ ]:
# Plot the image
plt.imshow(img, extent=backprojector.extent, interpolation="none");

Investigate Compton images with varying numbers of Compton cones.